## Exercise 04. Enrichment and transformations ##

## Imports ##

In [14]:
import pandas as pd
import numpy as np
# import requests

## Task ##

## 1. Read JSON ##
1. Прочитайте файл JSON, который вы сохранили в ex02.
+ Одна из колонок — число с плавающей точкой, поэтому зададим формат отображения в pandas с помощью `pd.options.display.float_format`: числа должны отображаться с двумя знаками после запятой.
+ В данных есть пропущенные значения по колонке Model; ничего с ними делать не нужно.

In [15]:
pd.options.display.float_format = '{:,.2f}'.format
df = pd.read_json('../data/auto.json')
df.head()

,CarNumber,Refund,Fines,Make,Model
0,Y163O8161RUS,2,"3,200.00",Ford,Focus
1,E432XX77RUS,1,"6,500.00",Toyota,Camry
2,7184TT36RUS,1,"2,100.00",Ford,Focus
3,X582HE161RUS,2,"2,000.00",Ford,Focus
4,92918M178RUS,1,"5,700.00",Ford,Focus


## 2. Enrich with a sample ##
2. Обогатите датафрейм, используя образец из этого датафрейма.
+ Создайте образец из 200 новых наблюдений с использованием `random_state=21`.
    + В образце не должно быть новых комбинаций номера автомобиля, марки и модели, чтобы весь набор данных оставался согласованным в этом отношении.
    + Нет ограничений по колонкам refund и fines. Вы можете случайным образом выбирать значения из этих колонок и применять их к любому номеру автомобиля.
+ Объедините образец с исходным датафреймом, чтобы создать новый датафрейм `concat_rows`.

In [16]:
print(df.count())
unique_cars = df[['CarNumber', 'Make', 'Model']].drop_duplicates()
sample_size = 200
random_param=21
sample = unique_cars.sample(n=sample_size, random_state=random_param)
unique_fines = df['Fines'].unique()
unique_refunds = df['Refund'].unique()
np.random.seed(random_param)
sample['Fines'] = np.random.choice(unique_fines, size=sample_size)
sample['Refund'] = np.random.choice(unique_refunds, size=sample_size)
concat_rows = pd.concat([df, sample])
print(concat_rows.count())

CarNumber    725
Refund       725
Fines        725
Make         725
Model        716
dtype: int64
CarNumber    925
Refund       925
Fines        925
Make         925
Model        912
dtype: int64


## 3. Enrich \`concat_rows\` ##
3. Обогатите датафрейм `concat_rows` новой колонкой с сгенерированными данными.
+ Создайте серию с именем "Year" с случайными целыми числами от 1980 до 2019.
+ Перед генерацией лет используйте `np.random.seed(21)`.
+ Объедините серию с датафреймом и задайте ей имя `fines`.

In [17]:
np.random.seed(random_param)
year = pd.Series(np.random.default_rng(seed=random_param).integers(low=1980, high=2019, size=len(concat_rows)))
fines = df.assign(Year=year)
fines

,CarNumber,Refund,Fines,Make,Model,Year
0,Y163O8161RUS,2,"3,200.00",Ford,Focus,1991
1,E432XX77RUS,1,"6,500.00",Toyota,Camry,2010
2,7184TT36RUS,1,"2,100.00",Ford,Focus,1995
3,X582HE161RUS,2,"2,000.00",Ford,Focus,2003
4,92918M178RUS,1,"5,700.00",Ford,Focus,1998
...,...,...,...,...,...,...
720,Y163O8161RUS,2,"1,600.00",Ford,Focus,1983
721,M0309X197RUS,1,"22,300.00",Ford,Focus,1984
722,O673E8197RUS,2,600.00,Ford,Focus,1997
723,8610T8154RUS,1,"2,000.00",Ford,Focus,2006


## 4. Enrich with anither dataframe ##
4. Обогатите датафрейм данными из другого датафрейма.
+ Создайте новый датафрейм с номерами автомобилей и их владельцами.
    + Возьмите самые популярные фамилии в США (вы можете найти файл `surname.json` в папке `datasets`).
    + Создайте новую серию с фамилиями, не содержащими специальных символов, таких как запятые или скобки. Количество фамилий должно быть равно количеству уникальных номеров автомобилей в образце (используйте `random_state=21`).
    + Создайте датафрейм `owners` с двумя колонками: `CarNumber` и `SURNAME`.

In [18]:
surnames = pd.read_json('../../datasets/surname.json')[0][1:]
owners = pd.DataFrame(fines['CarNumber'].unique(), columns=['CarNumber'])
surname_sample = surnames.sample(n=len(owners), random_state=random_param, replace=True).reset_index(drop=True)
owners = owners.assign(SURNAME=surname_sample)
owners

,CarNumber,SURNAME
0,Y163O8161RUS,RICHARDSON
1,E432XX77RUS,ROSS
2,7184TT36RUS,MORGAN
3,X582HE161RUS,BAILEY
4,92918M178RUS,LOPEZ
...,...,...
526,O136HO197RUS,CAMPBELL
527,O22097197RUS,HALL
528,M0309X197RUS,BAKER
529,O673E8197RUS,DIAZ


+ Добавьте ещё пять наблюдений в датафрейм `fines`. Придумайте свои идеи для CarNumber и других колонок.
+ Удалите последние 20 наблюдений из датафрейма `owners` и добавьте три новых наблюдения, которые не совпадают с добавленными в `fines`.

In [19]:
new_fines = pd.DataFrame({
    'CarNumber': ['A123BC456', 'B456DE789', 'C789FG012', 'D012HI345', 'E345JK678'],
    'Refund': [1, 2, 1, 2, 1],
    'Fines': ['1,500.00', '4,200.00', '3,300.00', '2,800.00', '5,500.00'],
    'Make': ['Honda', 'BMW', 'Mazda', 'Nissan', 'Subaru'],
    'Model': ['Civic', 'X5', '3', 'Altima', 'Impreza'],
    'Year': [2005, 2012, 2008, 2015, 2018]
})
fines = pd.concat([fines, new_fines], ignore_index=True)
fines.tail()

,CarNumber,Refund,Fines,Make,Model,Year
725,A123BC456,1,"1,500.00",Honda,Civic,2005
726,B456DE789,2,"4,200.00",BMW,X5,2012
727,C789FG012,1,"3,300.00",Mazda,3,2008
728,D012HI345,2,"2,800.00",Nissan,Altima,2015
729,E345JK678,1,"5,500.00",Subaru,Impreza,2018


In [20]:
owners = owners[:-20]
new_owners = pd.DataFrame({
    'CarNumber': ['Z999ZZ999', 'L123PP456', 'Q456ZZ789'],
    'SURNAME': ['SMITHS', 'JOHNSONS', 'WILLIAMSON']
})
owners = pd.concat([owners, new_owners], ignore_index=True)
owners.tail()

,CarNumber,SURNAME
509,O50197197RUS,WRIGHT
510,7608EE777RUS,HILL
511,Z999ZZ999,SMITHS
512,L123PP456,JOHNSONS
513,Q456ZZ789,WILLIAMSON


+ Объедините два датафрейма.
    + Новый датафрейм должен содержать только номера машин, которые есть в обоих датафреймах.
    + Новый датафрейм должен содержать все номера машин из обоих датафреймов.
    + Новый датафрейм должен содержать только номера машин из `fines`.
    + Новый датафрейм должен содержать только номера машин из `owners`.

In [21]:
new_df = pd.merge(fines, owners, on='CarNumber')
new_df

,CarNumber,Refund,Fines,Make,Model,Year,SURNAME
0,Y163O8161RUS,2,"3,200.00",Ford,Focus,1991,RICHARDSON
1,E432XX77RUS,1,"6,500.00",Toyota,Camry,2010,ROSS
2,7184TT36RUS,1,"2,100.00",Ford,Focus,1995,MORGAN
3,X582HE161RUS,2,"2,000.00",Ford,Focus,2003,BAILEY
4,92918M178RUS,1,"5,700.00",Ford,Focus,1998,LOPEZ
...,...,...,...,...,...,...,...
700,O68897197RUS,2,"12,300.00",Ford,Focus,1997,MITCHELL
701,O718MM163RUS,2,"8,594.59",Ford,Focus,1983,GREEN
702,7065C8197RUS,2,"11,400.00",Volkswagen,Passat,2012,BROWN
703,Y163O8161RUS,2,"1,600.00",Ford,Focus,1983,RICHARDSON


In [22]:
new_df = pd.merge(fines, owners, how='outer', on='CarNumber')
new_df

,CarNumber,Refund,Fines,Make,Model,Year,SURNAME
0,704687163RUS,2.00,"1,400.00",Ford,Focus,"1,981.00",ADAMS
1,704787163RUS,2.00,"2,800.00",Ford,Focus,"1,994.00",MORGAN
2,704987163RUS,2.00,"8,594.59",Ford,Focus,"1,984.00",MITCHELL
3,705287163RUS,2.00,"2,000.00",Ford,Focus,"1,988.00",GOMEZ
4,705387163RUS,2.00,700.00,Ford,Focus,"2,010.00",STEWART
...,...,...,...,...,...,...,...
728,Y969O8197RUS,2.00,"7,800.00",Ford,Focus,"2,014.00",LOPEZ
729,Y973O8197RUS,2.00,"8,594.59",Ford,Focus,"2,016.00",YOUNG
730,Y973O8197RUS,1.00,"34,800.00",Ford,Focus,"1,995.00",YOUNG
731,Y973O8197RUS,1.00,"69,600.00",Ford,Focus,"1,989.00",YOUNG


In [23]:
new_df = pd.merge(fines, owners, how='left', on='CarNumber')
new_df

,CarNumber,Refund,Fines,Make,Model,Year,SURNAME
0,Y163O8161RUS,2,"3,200.00",Ford,Focus,1991,RICHARDSON
1,E432XX77RUS,1,"6,500.00",Toyota,Camry,2010,ROSS
2,7184TT36RUS,1,"2,100.00",Ford,Focus,1995,MORGAN
3,X582HE161RUS,2,"2,000.00",Ford,Focus,2003,BAILEY
4,92918M178RUS,1,"5,700.00",Ford,Focus,1998,LOPEZ
...,...,...,...,...,...,...,...
725,A123BC456,1,"1,500.00",Honda,Civic,2005,NaN
726,B456DE789,2,"4,200.00",BMW,X5,2012,NaN
727,C789FG012,1,"3,300.00",Mazda,3,2008,NaN
728,D012HI345,2,"2,800.00",Nissan,Altima,2015,NaN


In [24]:
new_df = pd.merge(fines, owners, how='right', on='CarNumber')
new_df

,CarNumber,Refund,Fines,Make,Model,Year,SURNAME
0,Y163O8161RUS,2.00,"3,200.00",Ford,Focus,"1,991.00",RICHARDSON
1,Y163O8161RUS,2.00,"1,600.00",Ford,Focus,"1,983.00",RICHARDSON
2,E432XX77RUS,1.00,"6,500.00",Toyota,Camry,"2,010.00",ROSS
3,E432XX77RUS,2.00,"13,000.00",Toyota,Camry,"1,996.00",ROSS
4,7184TT36RUS,1.00,"2,100.00",Ford,Focus,"1,995.00",MORGAN
...,...,...,...,...,...,...,...
703,O50197197RUS,2.00,"7,800.00",Ford,Focus,"1,985.00",WRIGHT
704,7608EE777RUS,1.00,"4,000.00",Skoda,Octavia,"2,015.00",HILL
705,Z999ZZ999,NaN,NaN,NaN,NaN,NaN,SMITHS
706,L123PP456,NaN,NaN,NaN,NaN,NaN,JOHNSONS


## 5. Create a pivot table ##
5. Создайте сводную таблицу по датафрейму `fines`. Она должна выглядеть так (значения — суммы штрафов), но по всем годам. Значения могут отличаться.

In [25]:
fines.pivot_table(values='Fines', index=['Make', 'Model'], columns='Year', aggfunc='sum')

Year                     1980       1981       1982       1983       1984  \
Make       Model                                                            
BMW        X5             NaN        NaN        NaN        NaN        NaN   
Ford       Focus   345,100.00 111,600.00 102,589.17 213,389.17 139,289.17   
           Mondeo   46,200.00        NaN        NaN        NaN        NaN   
Honda      Civic          NaN        NaN        NaN        NaN        NaN   
Mazda      3              NaN        NaN        NaN        NaN        NaN   
Nissan     Altima         NaN        NaN        NaN        NaN        NaN   
Skoda      Octavia   1,800.00        NaN     600.00  10,594.59        NaN   
Subaru     Impreza        NaN        NaN        NaN        NaN        NaN   
Toyota     Camry          NaN        NaN  10,600.00   1,000.00        NaN   
           Corolla        NaN   7,600.00        NaN        NaN        NaN   
Volkswagen Golf           NaN        NaN        NaN        NaN        NaN   
           Jetta          NaN        NaN        NaN        NaN        NaN   
           Passat         NaN  12,800.00        NaN        NaN        NaN   
           Touareg        NaN        NaN        NaN        NaN        NaN   

Year                    1985      1986      1987       1988       1989  ...  \
Make       Model                                                        ...   
BMW        X5            NaN       NaN       NaN        NaN        NaN  ...   
Ford       Focus   85,000.00 87,594.59 92,694.59 157,489.17 221,500.00  ...   
           Mondeo        NaN       NaN       NaN        NaN        NaN  ...   
Honda      Civic         NaN       NaN       NaN        NaN        NaN  ...   
Mazda      3             NaN       NaN       NaN        NaN        NaN  ...   
Nissan     Altima        NaN       NaN       NaN        NaN        NaN  ...   
Skoda      Octavia  8,594.59  5,500.00    500.00   2,400.00  15,894.59  ...   
Subaru     Impreza       NaN       NaN       NaN        NaN        NaN  ...   
Toyota     Camry         NaN       NaN  1,000.00        NaN        NaN  ...   
           Corolla 16,000.00       NaN  6,900.00        NaN        NaN  ...   
Volkswagen Golf          NaN       NaN       NaN        NaN        NaN  ...   
           Jetta         NaN       NaN       NaN        NaN        NaN  ...   
           Passat        NaN       NaN  1,600.00   3,200.00        NaN  ...   
           Touareg       NaN       NaN       NaN        NaN        NaN  ...   

Year                    2009      2010       2011      2012      2013  \
Make       Model                                                        
BMW        X5            NaN       NaN        NaN  4,200.00       NaN   
Ford       Focus   75,289.17 91,889.17 101,600.00 52,700.00 75,600.00   
           Mondeo        NaN       NaN        NaN       NaN       NaN   
Honda      Civic         NaN       NaN        NaN       NaN       NaN   
Mazda      3             NaN       NaN        NaN       NaN       NaN   
Nissan     Altima        NaN       NaN        NaN       NaN       NaN   
Skoda      Octavia 10,594.59  1,000.00        NaN       NaN 63,300.00   
Subaru     Impreza       NaN       NaN        NaN       NaN       NaN   
Toyota     Camry         NaN 46,894.59        NaN       NaN       NaN   
           Corolla       NaN       NaN  22,700.00       NaN       NaN   
Volkswagen Golf          NaN       NaN        NaN       NaN       NaN   
           Jetta         NaN       NaN        NaN       NaN  1,000.00   
           Passat   2,500.00  8,594.59        NaN 11,400.00  8,594.59   
           Touareg  5,800.00       NaN        NaN       NaN       NaN   

Year                    2014       2015       2016      2017      2018  
Make       Model                                                        
BMW        X5            NaN        NaN        NaN       NaN       NaN  
Ford       Focus   53,794.59 100,789.17 120,194.59 65,089.17 77,089.17  
           Mondeo        NaN   2,200.00  

## 6. Save to CSV ##
6. Сохраните оба датафрейма `fines` и `owners` в CSV-файлы без индексов.

In [26]:
fines.to_csv('../data/fines.csv', index=False)
owners.to_csv('../data/owners.csv', index=False)